In [1]:
%autoawait
import time
import asyncio
import pprint

from aries_basic_controller.aries_controller import AriesAgentController

# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://bob-agent:8021"

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, mediation=True)


In [32]:
### Time to create an invitation

In [4]:
# Create Invitation
unmediated_invite = await agent_controller.connections.create_invitation()

Going to the mediation agent1 to generate and fetch the invitation

In [5]:
mediator_invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '13ee3314-a373-4207-8e93-2cf723f92d62', 'label': 'MEDIATOR1', 'serviceEndpoint': 'https://09aea288bf3c.ngrok.io', 'recipientKeys': ['6HKzVWtLQBd5zNTrb7EU11pVBjXN8euTKzRaiDFqjncg']}

In [6]:
response = await agent_controller.connections.accept_connection(mediator_invitation)
pp.pprint(response)

{   'accept': 'manual',
    'connection_id': '6eb45f74-39de-4784-ac0f-18b4ab3a1807',
    'created_at': '2021-03-31 12:58:19.600254Z',
    'invitation_key': '6HKzVWtLQBd5zNTrb7EU11pVBjXN8euTKzRaiDFqjncg',
    'invitation_mode': 'once',
    'my_did': 'RyB5D9K5EuZ7mYiRYNjR2W',
    'request_id': 'deccd541-e1e7-4225-bcee-cd58fd514691',
    'rfc23_state': 'request-sent',
    'routing_state': 'none',
    'state': 'request',
    'their_label': 'MEDIATOR1',
    'their_role': 'inviter',
    'updated_at': '2021-03-31 12:58:19.634305Z'}


In [8]:
connection_id = response["connection_id"]
print(connection_id)

6eb45f74-39de-4784-ac0f-18b4ab3a1807


In [ ]:

Now we will Request mediation

Now that we have successfully established a connection between Bob and the mediator agent1 we can proceed to request mediation from the mediator1.


In [9]:
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)
print(connection['state'])

active


In [11]:
# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection between the mediator agent and Bob's subwallet agent")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    print(mediation_req)


{'role': 'client', 'connection_id': '6eb45f74-39de-4784-ac0f-18b4ab3a1807', 'state': 'request', 'routing_keys': [], 'mediator_terms': [], 'updated_at': '2021-03-31 13:00:40.963923Z', 'mediation_id': 'e130d8c8-f71c-40f4-8ee0-99bbbdb02393', 'created_at': '2021-03-31 13:00:40.963923Z', 'recipient_terms': []}


Now we have to take a look at the mediation records and we should see our mediation in there

In [12]:
response = await agent_controller.mediation.get_mediation_records()
print(response)

[{'role': 'client', 'connection_id': '6eb45f74-39de-4784-ac0f-18b4ab3a1807', 'state': 'request', 'routing_keys': [], 'mediator_terms': [], 'updated_at': '2021-03-31 13:00:40.963923Z', 'mediation_id': 'e130d8c8-f71c-40f4-8ee0-99bbbdb02393', 'created_at': '2021-03-31 13:00:40.963923Z', 'recipient_terms': []}]



Here we are Setting a default mediator

By setting a default mediator, all DIDComm connections we establish will be routed through this mediator. To do this we use the appropriate API endpoint via the agent controller and pass it the mediation ID of our mediated connection.


In [13]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(response[0]['mediation_id'])
pp.pprint(default_mediation_res)

{   'connection_id': '6eb45f74-39de-4784-ac0f-18b4ab3a1807',
    'created_at': '2021-03-31 13:00:40.963923Z',
    'endpoint': 'https://09aea288bf3c.ngrok.io',
    'mediation_id': 'e130d8c8-f71c-40f4-8ee0-99bbbdb02393',
    'mediator_terms': [],
    'recipient_terms': [],
    'role': 'client',
    'routing_keys': ['58fVUfK2DpxMzBr8eCtHh425hYCmKo3EfwAbRwQwkxWk'],
    'state': 'granted',
    'updated_at': '2021-03-31 13:00:44.666639Z'}


In [ ]:

Quick Check to see our default mediator actully there

In [14]:
default_mediator = await agent_controller.mediation.get_default_mediator()
pp.pprint(default_mediator)

if default_mediator['connection_id'] != connection_id:
    print("Oooops! Something went wrong setting the default mediator. Please, check above and try again")
else:
    print("\n\n Hooray! We have succesfully set a default mediator.")


{   'connection_id': '6eb45f74-39de-4784-ac0f-18b4ab3a1807',
    'created_at': '2021-03-31 13:00:40.963923Z',
    'endpoint': 'https://09aea288bf3c.ngrok.io',
    'mediation_id': 'e130d8c8-f71c-40f4-8ee0-99bbbdb02393',
    'mediator_terms': [],
    'recipient_terms': [],
    'role': 'client',
    'routing_keys': ['58fVUfK2DpxMzBr8eCtHh425hYCmKo3EfwAbRwQwkxWk'],
    'state': 'granted',
    'updated_at': '2021-03-31 13:00:44.666639Z'}


 Hooray! We have succesfully set a default mediator.


In [15]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)
pp.pprint("\n" + invite_message['routingKeys'][0])

Connection ID 04c61a01-2a2f-47ab-b98e-9c0707da9879
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'b743a9cc-5bcf-4995-91e6-1155c140c90d', 'label': 'BOB', 'routingKeys': ['58fVUfK2DpxMzBr8eCtHh425hYCmKo3EfwAbRwQwkxWk'], 'recipientKeys': ['9Pr7jYhQGFWd3nfUGyai4t2xCRNnv9XHi284coNiLaq5'], 'serviceEndpoint': 'https://09aea288bf3c.ngrok.io'}
'\n58fVUfK2DpxMzBr8eCtHh425hYCmKo3EfwAbRwQwkxWk'


In [16]:
unmediated_invite_message = unmediated_invite['invitation']
pp.pprint(unmediated_invite_message)

{   '@id': 'ed7c98ab-7a5b-4ce3-a224-c664957c6cd0',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'BOB',
    'recipientKeys': ['3PhqVwU9QsGzdmX2bKV8mkcWevKszNjgCdrcKCSRdGTw'],
    'serviceEndpoint': 'https://372b9ed1adb7.ngrok.io'}


In [17]:

# Comparing endpoints to check and see the service endpoint
print("Unmediated endpoint: " + unmediated_invite_message['serviceEndpoint'] + "\n\n")
print("Mediated endpoint: " + invite_message['serviceEndpoint'])
print("Mediator Invitation Ednpoint: " + mediator_invitation['serviceEndpoint'])

Unmediated endpoint: https://372b9ed1adb7.ngrok.io


Mediated endpoint: https://09aea288bf3c.ngrok.io
Mediator Invitation Ednpoint: https://09aea288bf3c.ngrok.io


In [29]:
external_invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '4e41fff0-a57d-45e4-8ab1-ba5396e0f30e', 'serviceEndpoint': 'https://f7af63a23e7f.ngrok.io', 'recipientKeys': ['CWNEMw8ccGSQWqZZ3PVFp1kGgoTSZPM99X9d4EsekKYn'], 'label': 'ALICE'}

In [30]:
response = await agent_controller.connections.accept_connection(external_invitation)
pp.pprint(response)

{   'accept': 'manual',
    'connection_id': 'b82484e7-261d-4dc4-bfaf-983b4efedcd1',
    'created_at': '2021-03-31 13:30:05.050100Z',
    'invitation_key': 'CWNEMw8ccGSQWqZZ3PVFp1kGgoTSZPM99X9d4EsekKYn',
    'invitation_mode': 'once',
    'my_did': 'Prer3tu4N75o4ex713vnP7',
    'request_id': '7bc4852c-c2ce-4b0f-ba36-42ddaf6e5c6f',
    'rfc23_state': 'request-sent',
    'routing_state': 'none',
    'state': 'request',
    'their_label': 'ALICE',
    'their_role': 'inviter',
    'updated_at': '2021-03-31 13:30:05.099465Z'}


In [31]:
connection_id = response["connection_id"]
print(connection_id)

b82484e7-261d-4dc4-bfaf-983b4efedcd1


In [32]:
## get the connection
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)

{'their_role': 'inviter', 'connection_id': 'b82484e7-261d-4dc4-bfaf-983b4efedcd1', 'request_id': '7bc4852c-c2ce-4b0f-ba36-42ddaf6e5c6f', 'state': 'request', 'rfc23_state': 'request-sent', 'accept': 'manual', 'routing_state': 'none', 'invitation_mode': 'once', 'invitation_key': 'CWNEMw8ccGSQWqZZ3PVFp1kGgoTSZPM99X9d4EsekKYn', 'updated_at': '2021-03-31 13:30:05.099465Z', 'their_label': 'ALICE', 'created_at': '2021-03-31 13:30:05.050100Z', 'my_did': 'Prer3tu4N75o4ex713vnP7'}


In [34]:
# Let's check for the state
def check_connection(connection):
    if connection['state'] != 'active':
        print("No active connection. \n\nPlease go back and ensure you have established an active connection between the mediator agent and Bob's subwallet agent\n")    
        print("State: " + connection['state']+ "\n")    
    else:
        print("Active connection established\n")
        print("State: " + connection['state']+ "\n")    
        pp.pprint(connection)

check_connection(connection)

No active connection. 

Please go back and ensure you have established an active connection between the mediator agent and Bob's subwallet agent

State: request



In [27]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (external)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}

SyntaxError: EOL while scanning string literal (<ipython-input-27-a3b6f7d065ed>, line 12)

# Request Credential from Issuer

In [28]:
try:
    record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
    state = record['state']
    role = record['role']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
except ClientResponseError as err:
    print(err)

NameError: name 'ClientResponseError' is not defined

In [ ]:
# Store the credential
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

In [ ]:
try:
    response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
    state = response['state']
    role = response['role']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
except ClientResponseError as err:
    print(err)